In [2]:
import requests
import json
import os
import pandas
from IPython.display import display
import pandasql

In [3]:
api_key = os.environ.get("FRESH_DESK_API_KEY")
domain = "embraceio"
url = f"https://{domain}.freshdesk.com/api/v2/tickets"
response = requests.get(url=url, auth=(api_key, "x"))

In [4]:
if response.status_code == 200:
    print("Requests processed successfully")
    data = json.loads(response.content)
    # print(data)
else:
    print("Failed to read tickets")
    response = json.loads(response.content)
    print(response["errors"])

Requests processed successfully


# Pandasql: Allows you to use sql to query pandas dataframes: Doc ([sqldf for pandas](https://pythonrepo.com/repo/yhat-pandasql-python-data-containers))

In [ ]:
pysqldf = lambda q: pandasql.sqldf(q, globals())

In [ ]:
tickets_raw_df = pandas.DataFrame.from_dict(data)
tickets_raw_df

In [ ]:
tickets_df = tickets_raw_df[["id", "custom_fields", "created_at", "updated_at"]]
tickets_df = tickets_df.applymap(str)

In [ ]:
# Get custom fields values into seperate rows
custom_fields = tickets_raw_df["custom_fields"].tolist()
custom_fields_df = pandas.DataFrame(custom_fields, columns=["cf_responsible_engineer", "cf_account_type", "cf_app_id", "cf_main_cs_contact"])
custom_fields_df["id"] = tickets_raw_df["id"]
custom_fields_df = custom_fields_df.applymap(str)

In [ ]:
# Combine custom fields data (seperate rows) with tickets data
q = """
    SELECT 
          tickets_df.id
        , tickets_df.custom_fields
        , custom_fields_df.cf_responsible_engineer
        , custom_fields_df.cf_main_cs_contact
        , custom_fields_df.cf_account_type
        , custom_fields_df.cf_app_id
        , tickets_df.created_at
        , tickets_df.updated_at
    FROM tickets_df
    INNER JOIN custom_fields_df
        ON custom_fields_df.id = tickets_df.id
    ;
"""

combined_df = pysqldf(q)
display(combined_df)

# FreshDesk Groups

In [5]:
api_key = os.environ.get("FRESH_DESK_API_KEY")
domain = "embraceio"
url = f"https://{domain}.freshdesk.com/api/v2/groups?per_page=100"
response = requests.get(url=url, auth=(api_key, "x"))

In [ ]:
if response.status_code == 200:
    print("Requests processed successfully")
    data = json.loads(response.content)
    # print(data)
else:
    print("Failed to read tickets")
    response = json.loads(response.content)
    print(response["errors"])

Requests processed successfully


In [ ]:
groups_raw_df = pandas.DataFrame.from_dict(data)
groups_raw_df

,id,name,description,escalate_to,unassigned_for,business_hour_id,group_type,created_at,updated_at,auto_ticket_assign
0,66000168193,Customer Success,None,None,None,None,support_agent_group,2020-09-18T19:26:05Z,2021-10-06T16:13:14Z,0
1,66000168199,Engineering - BE,None,None,None,None,support_agent_group,2020-09-18T19:26:29Z,2021-08-02T15:50:26Z,0
2,66000429874,Engineering - FE,Frontend Team,None,None,None,support_agent_group,2021-04-15T18:16:28Z,2021-04-15T18:16:28Z,0
3,66000433581,Engineering - SDK,None,None,None,None,support_agent_group,2021-07-23T23:25:36Z,2021-10-06T16:20:02Z,0
4,66000168111,Product,None,None,None,None,support_agent_group,2020-09-18T18:46:19Z,2020-09-30T15:10:12Z,0
5,66000427634,Sales Engineer,None,None,None,None,support_agent_group,2021-02-23T16:42:05Z,2021-02-23T16:42:05Z,0
6,66000435712,Support - Customer Sucess,None,None,None,None,support_agent_group,2021-10-15T16:59:37Z,2021-10-19T11:02:15Z,0


# FD Status Mapping

- [How to get all the options for ticket status field through API?](https://community.developers.freshworks.com/t/how-to-get-all-the-options-for-ticket-status-field-through-api/1338)

In [6]:
api_key = os.environ.get("FRESH_DESK_API_KEY")
domain = "embraceio"
url = f'https://{domain}.freshdesk.com/api/v2/admin/ticket_fields/66000277594'
response = requests.get(url=url, auth=(api_key, "x"))

In [7]:
if response.status_code == 200:
    print("Requests processed successfully")
    data = json.loads(response.content)
    # print(data)
else:
    print("Failed to read tickets")
    response = json.loads(response.content)
    print(response["errors"])

Requests processed successfully


In [8]:
data

{'id': 66000277594,
 'name': 'status',
 'label': 'Status',
 'label_for_customers': 'Status',
 'position': 9,
 'type': 'default_status',
 'default': True,
 'customers_can_edit': False,
 'required_for_closure': False,
 'required_for_agents': True,
 'required_for_customers': False,
 'displayed_to_customers': True,
 'created_at': '2020-09-18T18:46:18Z',
 'updated_at': '2022-03-23T19:16:34Z',
 'archived': False,
 'choices': [{'id': 2,
   'label': 'Open',
   'label_for_customers': 'Being Processed',
   'value': 'Open',
   'stop_sla_timer': False,
   'default': True,
   'position': 1,
   'deleted': False},
  {'id': 8,
   'label': 'Backlog',
   'label_for_customers': 'This ticket has been Closed',
   'value': 'Backlog',
   'stop_sla_timer': True,
   'default': False,
   'position': 2,
   'deleted': False,
   'group_ids': [66000168111,
    66000168193,
    66000168199,
    66000429874,
    66000433581,
    66000435712,
    66000438079,
    66000427634,
    66000439756]},
  {'id': 9,
   'label':

In [9]:
status_raw_df = pandas.DataFrame.from_dict(data)
status_choices_df = pandas.DataFrame(status_raw_df["choices"].tolist(), columns=['id', 'value'])
status_choices_df

,id,value
0,2,Open
1,8,Backlog
2,9,Triage
3,11,Assigned to Product Team
4,10,Assigned to Eng Team - Triage
5,7,In progress
6,6,Waiting on Customer
7,12,Pending - CS
8,3,Pending
9,4,Resolved


# FD Priority Mapping

In [15]:
api_key = os.environ.get('FRESH_DESK_API_KEY')
domain = 'embraceio'
url = f'https://{domain}.freshdesk.com/api/v2/admin/ticket_fields/66000277595'
response = requests.get(url=url, auth=(api_key, "x"))

if response.status_code == 200:
    print("Requests processed successfully")
    data = json.loads(response.content)
    print(data)
else:
    print("Failed to read tickets")
    response = json.loads(response.content)
    print(response["errors"])

Requests processed successfully
{'id': 66000277595, 'name': 'priority', 'label': 'Priority', 'label_for_customers': 'Priority', 'position': 10, 'type': 'default_priority', 'default': True, 'customers_can_edit': True, 'required_for_closure': False, 'required_for_agents': True, 'required_for_customers': False, 'displayed_to_customers': True, 'created_at': '2020-09-18T18:46:18Z', 'updated_at': '2021-03-09T20:58:22Z', 'archived': False, 'choices': [{'label': 'Low', 'value': 1}, {'label': 'Medium', 'value': 2}, {'label': 'High', 'value': 3}, {'label': 'Urgent', 'value': 4}]}


In [18]:
priority_raw_df = pandas.DataFrame.from_dict(data)
priority_choices_df = pandas.DataFrame(priority_raw_df['choices'].tolist(), columns=['label', 'value'])
priority_choices_df

,label,value
0,Low,1
1,Medium,2
2,High,3
3,Urgent,4


# Eng FD On Call Tickets

In [9]:
api_key = os.environ.get("FRESH_DESK_API_KEY")
domain = "embraceio"
url = f'https://{domain}.freshdesk.com/api/v2/search/tickets?query="tag:"PRIORITY-ENG-FD-ONCALL" " '
response = requests.get(url=url, auth=(api_key, "x"))

In [10]:
if response.status_code == 200:
    print("Requests processed successfully")
    data = json.loads(response.content)
    # print(data)
else:
    print("Failed to read tickets")
    response = json.loads(response.content)
    print(response["errors"])

Requests processed successfully


In [4]:
data

[{'cc_emails': [],
  'fwd_emails': [],
  'reply_cc_emails': [],
  'ticket_cc_emails': [],
  'fr_escalated': False,
  'spam': False,
  'email_config_id': None,
  'group_id': 66000168193,
  'priority': 2,
  'requester_id': 66017283504,
  'responder_id': 66020186877,
  'source': 3,
  'company_id': 66000360237,
  'status': 2,
  'subject': 'Tala -Android - Snoozed a crash but still got an alert',
  'association_type': None,
  'support_email': None,
  'to_emails': None,
  'product_id': None,
  'id': 1508,
  'type': 'Feature Request',
  'due_by': '2021-11-25T15:08:34Z',
  'fr_due_by': '2021-11-05T23:08:34Z',
  'is_escalated': False,
  'custom_fields': {'cf_account_type': 'Customer',
   'cf_app_id': '/QsT3b',
   'cf_main_cs_contact': 'Harry Schaffer',
   'cf_in_notion_priority_ticket_db': True},
  'created_at': '2021-11-05T15:08:34Z',
  'updated_at': '2021-11-06T02:53:08Z',
  'associated_tickets_count': None,
  'tags': [],
  'internal_agent_id': None,
  'internal_group_id': None,
  'nr_due_by'